# Fetching data
As a first step toward calibrating an option pricing model, we aim to fetch data for our CIR++ model, as well as for the option pricing model. We therefore split our data sampling pipeline in 2:

    1. Sample data for fixed-income instruments, such as OIS, Swaptions, and Caps.
    2. Sample European option data.

We sample data using London Stock Exchange's (LSEG) Workspace terminal with Datastream Add-On. The data from the terminal is already clean, so we do not need to clean up the sample data. However, we note that options on equities listed on the American and Norwegian markets are American options, which are not of interest to us. We therefore sample data from the European Union (EU) market. The positive part with the market choice is that countries in the EU are all reliant on one central bank only, which essentially means that we can use the same estimates for the short rate in Germany, France, and any other country in the EU.

In [ ]:
import re
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

import yfinance as yf
import lseg.data as ld

#fixed-income
from lseg.data.content.ipa.financial_contracts import swap, swaption

In [6]:
ld.open_session()

<lseg.data.session.Definition object at 0x124838830 {name='workspace'}>

In [7]:
pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_rows', None)

# 1. Fetching Data for Fixed-Income Instruments

## 1.1 Overnight Indexed Swaps (OIS)

In [8]:
def fetch_OIS_quotes(tenor_periods : list, currency : str, OIS_TEMPLATE : str, VAL_DATE : str, START_DATE : str) -> pd.DataFrame:
    rate_fields = ['FixedRatePercent', 'ParRatePercent', 'FairRatePercent', 'SwapRatePercent', 'ForwardSwapRatePercent']
    base_fields = ['InstrumentDescription','LegDescription','InstrumentTag','LegTag', 'EndDate','Tenor','InterestType','DiscountCurveName','ForwardCurveName']
    fields = base_fields + rate_fields

    rows = []
    for tenor in tenor_periods:
        response = swap.Definition(
            instrument_tag = f'{currency} {tenor} swap',
            template = OIS_TEMPLATE,
            start_date = START_DATE,
            tenor = tenor,
            pricing_parameters=swap.PricingParameters(valuation_date=VAL_DATE),
            fields=fields
        )
        
        r = response.get_data().data.df.iloc[0]
        rows.append({
            'tenor': tenor,
            'rate_percent' : r.get('FixedRatePercent'),
            'instrument_description': r.get('InstrumentDescription'),
            'leg_description': r.get('LegDescription'),
            'discount_curve': r.get('DiscountCurveName'),
            'end': r.get('EndDate'),
            })
    
    df = pd.DataFrame(rows)
    df['START_DATE'] = START_DATE
    df['VAL_DATE'] = VAL_DATE
    return df

In [10]:
VAL_DATE = '2025-10-24'
START_DATE = '2025-10-24'

currency = 'EUR'
OIS_TEMPLATE = 'EURESTOIS' #'EUROIS'

OIS_TENOR_PERIODS = []
OIS_TENOR_PERIODS.append('1W')

for i in range(1,12):
    OIS_TENOR_PERIODS.append(f'{i}M')

OIS_TENOR_PERIODS.append('18M')

for i in range(1,31):
    OIS_TENOR_PERIODS.append(f'{i}Y')

OIS_DATA = fetch_OIS_quotes(OIS_TENOR_PERIODS, currency, OIS_TEMPLATE, VAL_DATE, START_DATE)
OIS_DATA.head()

,tenor,rate_percent,instrument_description,leg_description,discount_curve,end,START_DATE,VAL_DATE
0,1W,1.927702,Pay EUR Annual 1.93% vs Receive EUR Annual +0b...,Pay EUR Annual 1.93%,EUR - OIS ESTR,2025-10-31,2025-10-24,2025-10-24
1,1M,1.927277,Pay EUR Annual 1.93% vs Receive EUR Annual +0b...,Pay EUR Annual 1.93%,EUR - OIS ESTR,2025-11-24,2025-10-24,2025-10-24
2,2M,1.927994,Pay EUR Annual 1.93% vs Receive EUR Annual +0b...,Pay EUR Annual 1.93%,EUR - OIS ESTR,2025-12-24,2025-10-24,2025-10-24
3,3M,1.921542,Pay EUR Annual 1.92% vs Receive EUR Annual +0b...,Pay EUR Annual 1.92%,EUR - OIS ESTR,2026-01-26,2025-10-24,2025-10-24
4,4M,1.917252,Pay EUR Annual 1.92% vs Receive EUR Annual +0b...,Pay EUR Annual 1.92%,EUR - OIS ESTR,2026-02-24,2025-10-24,2025-10-24


In [ ]:
OIS_DATA.to_excel('OIS_DATA.xlsx')

## 1.2 Swaptions

In [12]:
def add_tenor(date_str: str, tenor: str) -> datetime:
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    m = re.match(r"^(\d+)([DWMY])$", tenor)

    if not m:
        raise ValueError(f"Bad tenor: {tenor}")
    n, u = int(m.group(1)), m.group(2)

    if u == 'D': delta = relativedelta(days=n)
    elif u == 'W': delta = relativedelta(weeks=n)
    elif u == 'M': delta = relativedelta(months=n)
    elif u == 'Y': delta = relativedelta(years=n)
    
    return dt + delta

In [ ]:
def fetch_OIS_ATM_swaption_grid(swap_tenor_periods: list, option_expiry_tenors: list, currency: str, OIS_TEMPLATE: str, VAL_DATE: str, settlement_type = swaption.SwaptionSettlementType.CASH, exercise_style = swaption.ExerciseStyle.EURO,  buy_sell = swaption.BuySell.BUY, payer_receiver = swaption.SwaptionType.PAYER, extra_fields: list = None, notional_amount: float = 1.0) -> pd.DataFrame:
    
    expiry_dates = [add_tenor(VAL_DATE, tenor).strftime('%Y-%m-%d') for tenor in option_expiry_tenors]
    
    base_fields = ['InstrumentTag', 'InstrumentDescription', 'EndDate', 'UnderlyingTenor', 'StrikePercent', 'ImpliedVolatilityPercent', 'PremiumPercent', 'ForwardPremiumPercent', 'DiscountCurveName', 'ForwardCurveName']
    fields = base_fields if extra_fields is None else list(dict.fromkeys(base_fields+extra_fields))

    rows = []
    for expiry in expiry_dates: 
        print(f'Expiry Date: {expiry}')
        for tenor in swap_tenor_periods:
            tag = f'{currency}_{expiry}_{tenor}_{'pay' if payer_receiver==swaption.SwaptionType.PAYER else "rec"}'

            swap_definition = swap.Definition(
                tenor=tenor,
                template=OIS_TEMPLATE,
                start_date=expiry,
                pricing_parameters=swap.PricingParameters(valuation_date=VAL_DATE)
            )
            
            swap_df = swap_definition.get_data().data.df

            Fwd_pct = None
            if(swap_df is not None and not swap_df.empty):
                for c in ["ForwardSwapRatePercent","ParRatePercent","FairRatePercent","SwapRatePercent"]:
                    if c in swap_df.columns:
                        Fwd_pct = float(swap_df[c].iloc[0])
                        break

            swaption_kwargs = dict(
                instrument_tag=tag, settlement_type=settlement_type, notional_amount=notional_amount,
                buy_sell=buy_sell, swaption_type=payer_receiver, end_date=expiry,
                exercise_style=exercise_style, underlying_definition=swap_definition,
                pricing_parameters=swaption.PricingParameters(valuation_date=VAL_DATE),
                fields=fields
            )

            if(Fwd_pct is not None):
                swaption_kwargs["strike_percent"] = Fwd_pct 

            df = swaption.Definition(**swaption_kwargs).get_data().data.df
            
            if(df is None or df.empty):
                rows.append({
                    'expiry': expiry,
                    'tenor': tenor,
                    'error': 'No Data Returned'
                })
                continue
            
            r = df.iloc[0]
            rows.append({
                'expiry': expiry,
                'tenor': tenor,
                'instrument_tag': r.get('InstrumentTag'),
                'description': r.get('InstrumentDescription'),
                'end_date': r.get('EndDate'),
                'strike_pct': r.get('StrikePercent'),
                'impvol_pct': r.get('ImpliedVolatilityPercent'),
                'premium_pct': r.get('PremiumPercent'),
                'fwd_premium_pct': r.get('ForwardPremiumPercent'),
                'discount_curve': r.get('DiscountCurveName'),
                'forward_curve': r.get('ForwardCurveName'),
                'vega_pct' : r.get('VegaPercent')
            })
    
    cols = ['expiry', 'tenor', 'instrument_tag', 'description', 'end_date', 'strike_pct', 'impvol_pct', 'vega_pct', 'premium_pct', 'fwd_premium_pct', 'discount_curve', 'forward_curve']
    SWPT_DATA = pd.DataFrame(rows)[cols]
    
    SWPT_DATA['VAL_DATE'] = VAL_DATE
    SWPT_DATA['START_DATE'] = START_DATE
    return SWPT_DATA

In [ ]:
option_expiry_tenors = [f'{i}Y' for i in range(1,16)]
swap_tenor_periods = [f'{i}Y' for i in range(1,16)]

extra_fields = ['VegaPercent', 'Vega', 'VegaBP', 'ForwardSwapRatePercent', 'SwapAnnuityPercent', 'OptionMaturityYearFraction']

print(swap_tenor_periods)

OIS_TEMPLATE = 'EUR_AB3E'
SWPT_DATA = fetch_OIS_ATM_swaption_grid(
    currency=currency,
    OIS_TEMPLATE=OIS_TEMPLATE,
    VAL_DATE=VAL_DATE,
    option_expiry_tenors=option_expiry_tenors,
    swap_tenor_periods=swap_tenor_periods,
    settlement_type=swaption.SwaptionSettlementType.CASH,
    exercise_style=swaption.ExerciseStyle.EURO,
    buy_sell=swaption.BuySell.BUY,
    payer_receiver=swaption.SwaptionType.PAYER,
    notional_amount=1.0,
    extra_fields=extra_fields
)

SWPT_DATA.head()

['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '11Y', '12Y', '13Y', '14Y', '15Y']
Expiry Date: 2026-10-24
Expiry Date: 2027-10-24
Expiry Date: 2028-10-24
Expiry Date: 2029-10-24
Expiry Date: 2030-10-24
Expiry Date: 2031-10-24
Expiry Date: 2032-10-24
Expiry Date: 2033-10-24
Expiry Date: 2034-10-24
Expiry Date: 2035-10-24
Expiry Date: 2036-10-24
Expiry Date: 2037-10-24
Expiry Date: 2038-10-24
Expiry Date: 2039-10-24
Expiry Date: 2040-10-24


,expiry,tenor,instrument_tag,description,end_date,strike_pct,impvol_pct,vega_pct,premium_pct,fwd_premium_pct,discount_curve,forward_curve,VAL_DATE,START_DATE
0,2026-10-24,1Y,EUR_2026-10-24_1Y_pay,Long payer EURO 1Y x 1Y Swaption,2026-10-24,2.135298,21.318497,0.383588,0.171169,0.174771,EUR EURIBOR 6M,EUR EURIBOR 6M,2025-10-24,2025-10-24
1,2026-10-24,2Y,EUR_2026-10-24_2Y_pay,Long payer EURO 1Y x 2Y Swaption,2026-10-24,2.227618,22.954064,0.756434,0.385862,0.393981,EUR EURIBOR 6M,EUR EURIBOR 6M,2025-10-24,2025-10-24
2,2026-10-24,3Y,EUR_2026-10-24_3Y_pay,Long payer EURO 1Y x 3Y Swaption,2026-10-24,2.313319,23.003767,1.121217,0.595017,0.607537,EUR EURIBOR 6M,EUR EURIBOR 6M,2025-10-24,2025-10-24
3,2026-10-24,4Y,EUR_2026-10-24_4Y_pay,Long payer EURO 1Y x 4Y Swaption,2026-10-24,2.387665,22.898198,1.476676,0.805045,0.821984,EUR EURIBOR 6M,EUR EURIBOR 6M,2025-10-24,2025-10-24
4,2026-10-24,5Y,EUR_2026-10-24_5Y_pay,Long payer EURO 1Y x 5Y Swaption,2026-10-24,2.456576,22.861418,1.822619,1.020631,1.042106,EUR EURIBOR 6M,EUR EURIBOR 6M,2025-10-24,2025-10-24


In [ ]:
SWPT_DATA.to_excel('swaption_data.xlsx')

In [ ]:
SWPT_VOL_MATRIX = SWPT_DATA.pivot_table(index='expiry', columns='tenor', values='impvol_pct', aggfunc='first')
display(SWPT_VOL_MATRIX)

# 2. Fecthing Data for Equity Options


## 2.1 Equity Options Prices

The option chains follow the given naming logic: `0#<RIC-root>*.<exchange>`, where `RIC-root` is the company abbreviation (i.e. `MUVGn` for Münchener Rückversicherung) and `exchange` identifies the place of the stock exchange at hand (i.e. `DE` for Germany).

In [ ]:
START_DATE_minus_1 = (pd.to_datetime(START_DATE) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")

tickers = [
    '^GDAXI',   # DAX index
    '^FCHI',    # French CAC 40 Index
    'MUV2.DE',  # Münchener Rückerversicherungsgesellschaft (Munich Re)
    'AIR.PA',   # Airbus SE
    'MC.PA',    # LVMH
    'ASML.AS',  # ASML (Amsterdam)
    'ALV.DE',   # Allianz
    'SAP.DE',   # SAP
    'CS.PA',    # AXA
    'SIE.DE',   # Siemens
    'SAN.MC',   # Banco Santander (Madrid)
    'RHM.DE',   # Rheinmetall
    'BNP.PA',   # BNP Paribas
]

RICs = ['0#GDAX*.EX', # German DAX index
        '0#FCHI*.p',   # French CAC 40 Index
        '0#MUVE*.EX', # Münchener Rückerversicherungsgesellschaft (Munich Re)
        '0#EADE*.EX', # Airbus SE
        '0#MOHE*.EX', # LVMH
        '0#ASME*.EX',  # ASML (Amsterdam)
        '0#ALVE*.EX',  # Allianz
        '0#SAPE*.EX',  # SAP
        '0#AXAE*.EX',  # AXA
        '0#SIEE*.EX',  # Siemens
        '0#SAN*.EX',    # Banco Santander (Madrid)
        '0#RHME*.EX',  # Rheinmetall
        '0#BNPE*.EX'   # BNP Paribas
] 
fields = ['PUTCALLIND', 'EXPIR_DATE', 'STRIKE_PRC', 'CF_CLOSE', 'VEGA', 'EXERCISESTYLE'] #'TR.OptionType' or 'EXERCISESTYLE'

for i, RIC in enumerate(RICs):
    print(RIC)
    df = ld.get_data(universe=RIC, fields=fields)
    df = df.drop(index=df.index[0]) #first row is data describing the given asset, nothing to do with option chain

    call_prices = df[df['PUTCALLIND'].str.strip().eq('CALL')].copy()
    call_prices.rename({'CF_CLOSE': 'price'})

    call_prices['time_to_maturity_years'] = (pd.to_datetime(call_prices['EXPIR_DATE']) - pd.Timestamp.now()) / pd.Timedelta(days=365)

    #extracting close prices for stocks using yfinance
    close_price_stock = yf.download(tickers[i], start=START_DATE_minus_1, end=START_DATE)
    print(close_price_stock)
    close_price_stock = float(close_price_stock['Close'].to_numpy()[0,0])

    call_prices['S0'] = round(close_price_stock, 5)

    call_prices.to_excel(f'{RIC}_option_prices.xlsx')